In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score, f1_score, auc
import matplotlib
import matplotlib.pyplot as plt
from ludwig.api import LudwigModel
import requests
import yaml
import json
from pathlib import Path
from typing import Dict, Any, Tuple
import plotly.graph_objects as go
import os
import shutil
import glob
from sdv.metadata import SingleTableMetadata


In [3]:
def read_data(data_location) -> pd.DataFrame:
    creditcard = pd.read_csv(data_location)
    return creditcard


In [41]:
# read data and isolate fraud transactions
df = read_data("G:/My Drive/Data-Centric Solutions/07. Blog Posts/kedro/data/creditcard.csv")

In [42]:
# add transaction ID
df['transaction_id'] = df.index
df["data"] = "real data"

In [43]:
df_fraud = df.loc[df["Class"] == 1]
df_fraud.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,data
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1,541,real data
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1,623,real data
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1,4920,real data
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1,6108,real data
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1,6329,real data


In [48]:
# detect metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_fraud)

metadata.update_column(
    column_name='transaction_id',
    sdtype='id')

# set primary key 
metadata.set_primary_key(column_name='transaction_id')
metadata.validate()
# code to inspect metadata
# python_dict = metadata.to_dict()
# python_dict

Detected metadata:
{
    "columns": {
        "Time": {
            "sdtype": "numerical"
        },
        "V1": {
            "sdtype": "numerical"
        },
        "V2": {
            "sdtype": "numerical"
        },
        "V3": {
            "sdtype": "numerical"
        },
        "V4": {
            "sdtype": "numerical"
        },
        "V5": {
            "sdtype": "numerical"
        },
        "V6": {
            "sdtype": "numerical"
        },
        "V7": {
            "sdtype": "numerical"
        },
        "V8": {
            "sdtype": "numerical"
        },
        "V9": {
            "sdtype": "numerical"
        },
        "V10": {
            "sdtype": "numerical"
        },
        "V11": {
            "sdtype": "numerical"
        },
        "V12": {
            "sdtype": "numerical"
        },
        "V13": {
            "sdtype": "numerical"
        },
        "V14": {
            "sdtype": "numerical"
        },
        "V15": {
            "sdtype": "

In [ ]:
# Pipeline

def read_data(data_location) -> pd.DataFrame:
    creditcard = pd.read_csv(data_location)
    return creditcard


def split_data(creditcard: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:

    train_df, holdout_df = train_test_split(creditcard, test_size=0.2, random_state=42)
    
    return train_df, holdout_df


def run_experiment(train_df: pd.DataFrame, model_yaml, output_dir) -> pd.DataFrame:

    # Send a GET request to the URL
    response = requests.get(model_yaml)

    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Load the YAML data from the response text
    config = yaml.safe_load(response.text)

    # Set up your experiment
    model = LudwigModel(config=config)
    experiment_results = model.experiment(
      dataset=train_df,
      output_directory=output_dir
    )
    
    df = pd.DataFrame()
    
    delete_file()

    
    return exp_run


def run_predictions(holdout_df: pd.DataFrame, exp_run: pd.DataFrame, output_dir) -> pd.DataFrame:
    
    # dummpy input varibale
    df = exp_run
    
    latest_experiment_dir = get_latest_experiment_dir(output_dir)
    model_path = Path(latest_experiment_dir) / 'model'

    # Load the model
    model = LudwigModel.load(model_path)
    
    # run predictions on holdout
    predictions, _ = model.predict(dataset=holdout_df)
    
    full_predictions = predictions.merge(right=holdout_df,   left_index=True, right_index=True)
    full_predictions['Class_predictions'] = full_predictions['Class_predictions'].map({True: 1, False: 0})
    
    return full_predictions


def model_training_diagnostics(full_predictions: pd.DataFrame, output_dir) -> Tuple[matplotlib.figure.Figure, go.Figure]:
    
    # plot roc curve 
    
    # plot roc curve 
    fpr, tpr, thresholds = roc_curve(full_predictions['Class'], full_predictions['Class_predictions'])
    roc_auc = auc(fpr, tpr)

    # Create the base figure
    fig = go.Figure()

    # Add the ROC curve
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC curve (area = {roc_auc:.2f})'))

    # Add the random guess line
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guess', line=dict(dash='dash')))

    # Update the layout
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=700,
        title='Receiver Operating Characteristic'
    )

    roc_curve_plot = fig
        
    # plot loss curve
    latest_experiment_dir = get_latest_experiment_dir(output_dir)

    json_path = latest_experiment_dir + "/training_statistics.json"

    # Load the JSON file
    with open(json_path, 'r') as f:
        train_stats = json.load(f)

    train_loss = train_stats['training']['Class']['loss']
    validation_loss = train_stats['validation']['Class']['loss']
    test_loss = train_stats['test']['Class']['loss']

    # Create list of epochs
    epochs = list(range(1, len(train_loss) + 1))

    # Create the plot
    fig = go.Figure()

    # Add traces
    fig.add_trace(go.Scatter(x=epochs, y=train_loss, mode='lines', name='Training loss'))
    fig.add_trace(go.Scatter(x=epochs, y=validation_loss, mode='lines', name='Validation loss'))
    fig.add_trace(go.Scatter(x=epochs, y=test_loss, mode='lines', name='Test loss'))

    # Add details
    fig.update_layout(title='Training, Validation and Test Loss', xaxis_title='Epochs', yaxis_title='Loss')
    
    loss_plot = fig
    
    return loss_plot, roc_curve_plot